# Debugging simulation failures

**Objective:** Demonstrate common simulation failures and give some hints for interpreting, debugging, and fixing them.

In [ ]:
%matplotlib inline
import os
import amici
from amici.petab_import import import_petab_problem
from amici.petab_objective import simulate_petab, RDATAS, EDATAS
from amici.plotting import plot_state_trajectories, plot_jacobian
import petab
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from contextlib import suppress

try:
    import benchmark_models_petab
except ModuleNotFoundError:
    # install `benchmark_models_petab` if necessary
    %pip install -q -e "git+https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab.git@master#subdirectory=src/python&egg=benchmark_models_petab"
    try:
        import benchmark_models_petab
    except ModuleNotFoundError:
        print("** Please restart the kernel. **")

## Overview

In the following, we will simulate models contained in the [PEtab Benchmark Collection](https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab/) to demonstrate a number of simulation failures to analyze and fix them. We use the PEtab format, as it makes model import and simulation much easier, but everything illustrated here, also applies to plain SBML or PySB import.

Note that, due to numerical issues, the examples below may not be fully reproducible on every system.

If any simulation failures occur, they will be printed via Python logging. 

Programmatically, simulation success can be checked via `ReturnDataView.status`. In case of a successful simulation, and only then, this value corresponds to `amici.AMICI_SUCCESS`.
In case of a simulation error, all quantities in `ReturnData`/`ReturnDataView` will be reported up to the time of failure, the rest will be `NaN`. The likelihood and it's gradient will always be `NaN` in case of failure.

## `AMICI_TOO_MUCH_WORK` - `mxstep steps taken before reaching tout`

Let's run a simulation:


In [ ]:
petab_problem = benchmark_models_petab.get_problem("Fujita_SciSignal2010")
amici_model = import_petab_problem(petab_problem, verbose=False, force_compile=False)

np.random.seed(2991)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_TOO_MUCH_WORK', 'AMICI_SUCCESS', 'AMICI_SUCCESS']

**What happened?**

AMICI failed to integrate the forward problem. The problem occurred for only one simulation condition, `condition_step_03_0`. The issue occurred at $t = 3031.8$, where the CVODES reached the maximum number of steps.

**How to address?**

The number of steps the solver has to take is closely related to the chosen error tolerance. More accurate results, more steps. Therefore, this problem can be solved in two ways:

1. Increasing the maximum number of steps via [`amici.Solver.setMaxSteps`](https://amici.readthedocs.io/en/latest/generated/amici.amici.Solver.html#amici.amici.Solver.setMaxSteps). Note that this will increase the time required for simulation, and that simulation may still fail eventually. Sometimes it may be preferable to not increase this limit but rather fail fast. Also note that increasing the number of allowed steps increase RAM requirements (even if fewer steps are actually taken), so don't set this to ridiculously large values in order to avoid this error.

2. Reducing the number of steps CVODES has to take. This is determined by the required error tolerance. There are various solver error tolerances than can be adjusted. The most relevant ones are those controlled via [`amici.Solver.setRelativeTolerance()`](https://amici.readthedocs.io/en/latest/generated/amici.amici.Solver.html#amici.amici.Solver.setRelativeTolerance) and [`amici.Solver.setAbsoluteTolerance()`](https://amici.readthedocs.io/en/latest/generated/amici.amici.Solver.html#amici.amici.Solver.setAbsoluteTolerance).

So, let's fix that:

In [ ]:
# let's increase the allowed number of steps by 10x:
print("Increasing allowed number of steps ...")
amici_solver = amici_model.getSolver()
amici_solver.setMaxSteps(10 * amici_solver.getMaxSteps())

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver
)

print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])
print("Simulations finished succesfully.")
print()


# let's relax the relative error tolerance by a factor of 50
print("Relaxing relative error tolerance ...")
amici_solver = amici_model.getSolver()
amici_solver.setRelativeTolerance(50 * amici_solver.getRelativeTolerance())

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])
print("Simulations finished succesfully.")


## `Internal t = [...] and h = [...] are such that t + h = t on the next step`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Crauste_CellSystems2017")
amici_model = import_petab_problem(petab_problem, verbose=False)

np.random.seed(1)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_TOO_MUCH_WORK']

**What happened?**

The forward simulation failed because AMICI the solver exceeded the maximum number of steps. Unlike in the previous case of  `mxstep steps taken before reaching tout` (see above), here we got several additional warnings that the current step size $h$ is numerically zero.

**How to address?**

The warning `Internal t = [...] and h = [...] are such that t + h = t on the next step` tells us that the solver is not able to move forward. The solver may be able to recover from that, but not always.

Let's look at the state trajectories to see what's going on. Such a tiny step size is usually related to very fast dynamics. We repeat the simulation with additional timepoints before the point of failure:

In [ ]:
# Create a copy of this simulation condition
edata = amici.ExpData(res[EDATAS][0])
edata.setTimepoints(np.linspace(0, 0.33011, 5000))
amici_solver = amici_model.getSolver()
rdata = amici.runAmiciSimulation(amici_model, amici_solver, edata)

# Visualize state trajectories
plot_state_trajectories(rdata, model=amici_model)
plt.yscale("log")

We can see a steep increase for `Pathogen` just before the error occurs. Let's zoom in:

In [ ]:
plt.plot(rdata.t, rdata.by_id("Pathogen"))
plt.xlabel("time")
plt.ylabel("Pathogen");

The solver is unable to handle such a steep increase. There is not much we can do. Increasing the tolerances will let the solver proceed a bit further, but this is usually not enough. Most likely there is a problem in the model or in the choice of parameter values.

## `the error test failed repeatedly or with |h| = hmin`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Fujita_SciSignal2010")
amici_model = import_petab_problem(petab_problem, verbose=False)

np.random.seed(4920)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])

assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_SUCCESS', 'AMICI_ERR_FAILURE', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS']

**What happened?**

AMICI failed to integrate the forward problem. The problem occurred for only one simulation condition, `condition_step_00_3`. The issue occurred at $t = 429.232$, where the error test failed.
This means, the solver is unable to take a step of non-zero size without violating the choosen error tolerances.

**How to address?**

The step size is computed based on the Jacobian. Inspecting `ReturnData.J` shows us that we have rather large values in the Jacobian:

In [ ]:
rdata = res[RDATAS][1]

# Show Jacobian as heatmap
plot_jacobian(rdata)

print(f"largest absolute Jacobian value: {np.max(np.abs(rdata.J)):.3g}")

In this case, the default relative error tolerance may be too high and lead too large absolute errors. 

Let's retry simulation using stricter tolerances:

In [ ]:
# set stricter relative error tolerance
amici_solver = amici_model.getSolver()
amici_solver.setRelativeTolerance(amici_solver.getRelativeTolerance() / 10)

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])
print("Simulations finished succesfully.")

## `Cvode routine CVode returned a root after reinitialization`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Weber_BMC2015")
amici_model = import_petab_problem(petab_problem, verbose=False, force_compile=False)

np.random.seed(4)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_ERROR', 'AMICI_SUCCESS']

**What happened?**

The simulation failed because the initial step-size after an event or heaviside function was too small. The error occured during simulation of condition `model1_data1` after successful preequilibration (`model1_data2`).

**How to address?**

The error message already suggests a fix for this situation, so let's try increasing the relative tolerance:

In [ ]:
amici_solver = amici_model.getSolver()
amici_solver.setRelativeTolerance(200 * amici_solver.getRelativeTolerance())

np.random.seed(4)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])

## `AMICI encountered a NaN / Inf value for [...]`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Borghans_BiophysChem1997")
amici_model = import_petab_problem(petab_problem, verbose=False)

np.random.seed(18)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_FIRST_RHSFUNC_ERR']

**What happened?**

The forward simulation failed because AMICI encountered a `NaN` value when simulating condition `model1_data1`.
Then `NaN`s occurred in $\dot x$ and $w$ (model expressions, such as reaction fluxes or assignment rules). Furthermore, the failure occurred at the first call, so at $t = t_0$ (here: $t = 0$).

**How to address?**

The `NaN` in $\dot x$ is most likely a consequence of the one in $w$. (A subset of) the dependency tree looks something like:

[![](https://mermaid.ink/img/pako:eNpdkrFuwyAQhl8FIWVL1MwMndKFtd1wBmJIg2IDwucCivLuxQp2zvaA_P1398Od7kFbpzRl9DdIfyM_p8aS8t3FnZHW2QGkheH8Er3wjHgZZK9Bh1kFAYyA6XXldBTpyIixBozsSHGAJSQSWwlccEa4bN3FSFu1KCIjOvmgh8GUF8y1yoGYDkZU-lBQ5SwyI-4y6PAnL52esl-B3a68pDZDDofPhfyKYEVLacSVERdGXFchzYAu59iBYweOHTh2qBAxvLupI3s9eJ41pndqGdOqvYXThv2G7xuOiBf7jHMzNsr41oyvzNgv0z3tdeilUWXzHlOooXDTvW4oK79KX-XYQUMb-yypcgT3nW1LGYRR7-noVVmhk5FlZ3vKrrIbFvVLGXChis9_j9jNUw?type=png)](https://mermaid.live/edit#pako:eNpdkrFuwyAQhl8FIWVL1MwMndKFtd1wBmJIg2IDwucCivLuxQp2zvaA_P1398Od7kFbpzRl9DdIfyM_p8aS8t3FnZHW2QGkheH8Er3wjHgZZK9Bh1kFAYyA6XXldBTpyIixBozsSHGAJSQSWwlccEa4bN3FSFu1KCIjOvmgh8GUF8y1yoGYDkZU-lBQ5SwyI-4y6PAnL52esl-B3a68pDZDDofPhfyKYEVLacSVERdGXFchzYAu59iBYweOHTh2qBAxvLupI3s9eJ41pndqGdOqvYXThv2G7xuOiBf7jHMzNsr41oyvzNgv0z3tdeilUWXzHlOooXDTvW4oK79KX-XYQUMb-yypcgT3nW1LGYRR7-noVVmhk5FlZ3vKrrIbFvVLGXChis9_j9jNUw)

Always look for the most basic (furthest up) model quantities.
In cases where there non-finite values occur in expressions further down, rerunning the simulation after calling `Model.setAlwaysCheckFinite(True)` may give some further hints on where the issue originates.

The `NaN` in $w$ occurred for `flux_v7_v_6` (see error log), i.e., when computing the reaction flux for reaction `v7_v_6`. As $w$ only depends on $(t, p, k, x)$ and no non-finite values have been reported for those, the issue has to be in the respective flux equation.

Let's look at that expression. This can either be done by inspecting the underlying SBML model (e.g., using COPASI), or by checking the generated model code:

In [ ]:
# model name and source code location
model_name = amici_model.module.__package__
model_src_dir = Path(amici_model.module.__file__).parents[1]

# find the problematic expression in the model source code
!grep flux_v7_v_6 {model_src_dir}/{model_name}_w.cpp

What could go wrong? We can obtain `NaN` from any of these symbols symbols being `NaN`, or through division by zero.

Let's let's check the denominator first: $$(A\_state^2 + Kp^2)*(Kd^{n\_par} + Z\_state^{n\_par})$$


`A_state` and `Z_state` are state variables, `Kd`, `K_p`, and `n_par` are parameters.

As the error occurred at $t = t_0$, let's ensure the initial state is non-zero and finite:

In [ ]:
rdata = res[RDATAS][0]
edata = res[EDATAS][0]
# check initial states
x0 = dict(zip(amici_model.getStateIds(), rdata.x0))
print(f"{x0=}")

The initial states are fine - the first multiplicand is non-zero, as $x_0$ was non-zero. 

So let's check the parameter values occurring in the second multiplicand:

In [ ]:
# we have to account for the chosen parameter scale
from itertools import starmap
unscaled_parameter = dict(zip(
    amici_model.getParameterIds(),
    starmap(amici.getUnscaledParameter, zip(edata.parameters, edata.pscale)),
))
print(dict((p, unscaled_parameter[p]) for p in ('Kd', 'Kp', 'n_par')))

Considering that `n_par` occurrs as exponent, it's magnitude looks pretty high.
This term is very likely causing the problem - let's check:

In [ ]:
print(f"{x0['Z_state']**unscaled_parameter['n_par'] + unscaled_parameter['Kd']**unscaled_parameter['n_par']=}")

Indeed, no way we can fix this for the given model.
This was most likely an unrealistic parameter value, originating from a too high upper parameter bound for `n_par`.
Therefore, if this error occurs during optimization, a first step could be adapting the respective parameter bounds.
In other cases, this may be a result of unfortunate arrangement of model expressions, which can sometimes be solved by passing a suitable simplification function to the model import.

<a id='unsuccessful_factorization'></a>

## `Steady state sensitivity computation failed due to unsuccessful factorization of RHS Jacobian`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Blasi_CellSystems2016")
with suppress(KeyError):
    del os.environ["AMICI_EXPERIMENTAL_SBML_NONCONST_CLS"]
amici_model = import_petab_problem(
    petab_problem, 
    verbose=False,
    force_compile=True,
    model_name="Blasi_CellSystems2016_1"
)

amici_solver = amici_model.getSolver()
amici_solver.setSensitivityMethod(amici.SensitivityMethod.forward)
amici_solver.setSensitivityOrder(amici.SensitivityOrder.first)

np.random.seed(1474)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_ERROR']

**What happened?**

AMICI failed to compute steadystate sensitivities, because it was not able to factorize the Jacobian.

**How to address?**

This is most likely a result of a singular Jacobian. Let's check the condition number:

In [ ]:
rdata = res[RDATAS][0]
np.linalg.cond(rdata.J)

Indeed, the condition number shows that the Jacobian is numerically singular. If this happens consistently, it is usually due to conserved quantities in the model.

There are two ways we can address that:

1. Use numerical integration to compute sensitivities, for which a singular Jacobian is not an issue. This is, usually, slower, though.
2. Remove any conserved quantities.

Let's try both approaches:

In [ ]:
# use numerical integration
amici_model.setSteadyStateSensitivityMode(amici.SteadyStateSensitivityMode.integrationOnly)

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])

In [ ]:
# Remove conserved quantities - this requires re-importing the model

# this is enabled by the `AMICI_EXPERIMENTAL_SBML_NONCONST_CLS` environment variable
os.environ["AMICI_EXPERIMENTAL_SBML_NONCONST_CLS"] = "1"
amici_model = import_petab_problem(
    petab_problem, 
    verbose=False,
    # we need a different model name if we import the model again
    # we cannot load a model with the same name as an already loaded model
    model_name="Blasi_CellSystems2016_2"
    force_compile=True,
)
del os.environ["AMICI_EXPERIMENTAL_SBML_NONCONST_CLS"]

amici_solver = amici_model.getSolver()
amici_solver.setSensitivityMethod(amici.SensitivityMethod.forward)
amici_solver.setSensitivityOrder(amici.SensitivityOrder.first)

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])

## `Steady state computation failed`

Let's run a simulation:

In [ ]:
petab_problem = benchmark_models_petab.get_problem("Brannmark_JBC2010")
amici_model = import_petab_problem(
    petab_problem, 
    verbose=False,
)

amici_solver = amici_model.getSolver()

np.random.seed(1851)
problem_parameters = dict(
    zip(
        petab_problem.x_free_ids,
        petab_problem.sample_parameter_startpoints(n_starts=1)[0],
    )
)
res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("Status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_ERROR', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS']

**What happened?**

All given experimental conditions require pre-equilibration, i.e., finding a steady state. AMICI first tries find a steady state using the Newton solver, if that fails, it tries simulating until steady state, if that also failes, it tries the Newton solver from the end of the simulation. In this case, all three failed. Neither Newton's method nor simulation yielded a steadystate satisfying the required tolerances.

This can also be seen in `ReturnDataView.preeq_status` (the three statuses corresponds to Newton \#1, Simulation, Newton \#2):

In [ ]:
rdata = res[RDATAS][0]
list(map(amici.SteadyStateStatus, rdata.preeq_status.flatten()))

**How to address?**

There are several ways to address that:

1. Stricter integration tolerances (preferred if affordable - higher accuracy, but generally slower)

2. Looser steadystate tolerances (lower accuracy, generally faster)

3. Increase the number of allowed steps for Newton's method

Let's try all of them:

In [ ]:
# Reduce relative tolerance for integration by a factor of ten
amici_solver = amici_model.getSolver()
amici_solver.setRelativeTolerance(1/10 * amici_solver.getRelativeTolerance())

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])

rdata = res[RDATAS][0]
print(f"preeq_status={list(map(amici.SteadyStateStatus, rdata.preeq_status.flatten()))}")
print(f"{rdata.preeq_numsteps=}")
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])

In [ ]:
# Increase relative tolerance
for log10_relaxation_factor in range(1, 10):
    print(f"Relaxing tolerances by factor {10 ** log10_relaxation_factor}")
    amici_solver = amici_model.getSolver()
    amici_solver.setRelativeToleranceSteadyState(amici_solver.getRelativeToleranceSteadyState() * 10 ** log10_relaxation_factor)
    
    res = simulate_petab(
        petab_problem=petab_problem, 
        amici_model=amici_model,
        problem_parameters=problem_parameters,
        scaled_parameters=True,
        solver=amici_solver,
    )
    if all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS]):
        print(f"-> Succeeded with relative steady state tolerance {amici_solver.getRelativeToleranceSteadyState()}\n")
        break
    else:
        print("-> Failed.\n")

print("status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])

rdata = res[RDATAS][0]
print(f"preeq_status={list(map(amici.SteadyStateStatus, rdata.preeq_status.flatten()))}")
print(f"{rdata.preeq_numsteps=}")
assert all(rdata.status == amici.AMICI_SUCCESS for rdata in res[RDATAS])

That fixed the error, and took only a quarter of the number steps as the previous run, but at the cost of much lower accuracy.

In [ ]:
# Let's try increasing the number of Newton steps
# (this is 0 by default, so the Newton solver wasn't used before, 
#  as can be seen from the 0 in `rdata.preeq_numsteps[0]`)
amici_solver = amici_model.getSolver()
amici_solver.setNewtonMaxSteps(10**4)

res = simulate_petab(
    petab_problem=petab_problem, 
    amici_model=amici_model,
    problem_parameters=problem_parameters,
    scaled_parameters=True,
    solver=amici_solver,
)
print("status:", [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]])

rdata = res[RDATAS][0]
print(f"preeq_status={list(map(amici.SteadyStateStatus, rdata.preeq_status.flatten()))}")
print(f"{rdata.preeq_numsteps=}")
assert [amici.simulation_status_to_str(rdata.status) for rdata in res[RDATAS]] == ['AMICI_ERROR', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS', 'AMICI_SUCCESS']

Increasing the maximum number of Newton steps doesn't seem to help here. The Jacobian was numerically singular and its factorization failed. This can be a result of conserved quantities in the model. Section [`Steady state sensitivity computation failed due to unsuccessful factorization of RHS Jacobian`](#unsuccessful_factorization) shows how to address that.